# Summary Statistics
## Basic Network Analysis
In this notebook the network is analyzed with simple measures. This is necessary for understanding the basic properties of the network.

In [1]:
import os
import pandas as pd
import networkx as nx
import numpy as np
import itertools
import json
from collections import Counter

# 1. Data

Following Ville's notebook. Had to do some minor edits because the data had some unexpected errors

Load data:

In [4]:
path = str(os.path.abspath(os.path.join('',"./")))
data = pd.read_json(path + "/ksw_full_372_30min_25-03-2020.json",orient="records", dtype=False, lines=True, encoding="utf-8")

ValueError: Expected object or value

In [4]:
#data.head()

In [5]:
data.shape

(91796, 38)

In [6]:
data.dtypes

created_at                   datetime64[ns, UTC]
id                                       float64
id_str                                    object
text                                      object
source                                    object
truncated                                 object
in_reply_to_status_id                    float64
in_reply_to_status_id_str                 object
in_reply_to_user_id                      float64
in_reply_to_user_id_str                   object
in_reply_to_screen_name                   object
user                                      object
geo                                       object
coordinates                               object
place                                     object
contributors                             float64
retweeted_status                          object
is_quote_status                           object
quote_count                              float64
reply_count                              float64
retweet_count       

In [7]:
# clean data
#data = data[pd.notnull(data['id_str'])]
#data[data.apply(pd.DataFrame.to_dict).notna()].dropna()
#data.apply(pd.DataFrame.to_dict, orient = 'split')

#[0 if type(row)!= pd.core.series.Series else 1 for _,row in data.iterrows()]
#data['user']

Restructure:

In [8]:
data['id_str'] = data['id_str'].astype(str)
data['in_reply_to_user_id_str'] = data['in_reply_to_user_id_str'].astype(str)

In [15]:
data['user_id_str']           = [row['id_str'] if type(row) != float else None for row in data['user']]
data['user_name']             = [row['name'] if type(row) != float else None for row in data['user']]
data['user_screen_name']      = [row['screen_name'] if type(row) != float else None for row in data['user']]
data['user_defined_location'] = [row['location'] if type(row) != float else None for row in data['user']]
data['user_followers_count']  = [row['followers_count'] if type(row) != float else None for row in data['user']]
data['place_country_code']    = [row['country_code'] if row is not None and type(row) != float else None for row in data['place']]
data['place_name']            = [row['name'] if row is not None and type(row)!= float else None for row in data['place']]
data['place_type']            = [row['place_type'] if row is not None and type(row) != float else None for row in data['place']]
data['place_coordinates']     = [dict(row['bounding_box'])['coordinates'][0][0] if row is not None and type(row) != float else None for row in data['place']]
data['hashtags']              = [[htags['text'].lower() if type(row) != float else None for htags in row['hashtags']] if type(row) != float else None for row in data['entities']]
data['user_location']         = [row['location'] if type(row) != float else None for row in data['user']]
data['user_created_at']       = [row['created_at'] if type(row) != float else None for row in data['user']]
data['user_geo_enabled']      = [row['geo_enabled']  if type(row) != float else None for row in data['user']]
data['is_retweet']            = [1 if b else 0 for b in data['retweeted_status'].notnull()]

In [16]:
data[pd.notnull(data['id_str'])]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_followers_count,place_country_code,place_name,place_type,place_coordinates,hashtags,user_location,user_created_at,user_geo_enabled,is_retweet
0,2020-03-25 09:32:54+00:00,1.242746e+18,1242746271892267009,RT @dumbricardo: dont mind me im just over he...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,349.0,None,None,None,None,[],08-25-18 7:32PM,Mon Sep 07 01:06:22 +0000 2015,True,1
1,2020-03-25 09:32:54+00:00,1.242746e+18,1242746271917637633,RT @SteveScalise: RT to tell Nancy Pelosi that...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,376.0,None,None,None,None,[],Portsmouth NH,Wed May 26 15:57:51 +0000 2010,True,1
2,2020-03-25 09:32:55+00:00,1.242746e+18,1242746271976370176,RT @liamgallagher: Listen up Brothers and Sist...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,57.0,None,None,None,None,[],"Cork, Ireland",Tue Sep 26 23:09:29 +0000 2017,False,1
3,2020-03-25 09:32:55+00:00,1.242746e+18,1242746272018124802,RT @TrendRamCharan: When the powers of opposin...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,51.0,None,None,None,None,[],None,Sat Dec 28 13:59:34 +0000 2019,False,1
4,2020-03-25 09:32:55+00:00,1.242746e+18,1242746272102170624,RT @MayUbeku: For those blackmailing others wi...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,385.0,None,None,None,None,[],"Lagos, Nigeria",Sun Apr 08 18:53:20 +0000 2012,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91791,2020-03-25 10:02:54+00:00,1.242754e+18,1242753817617317888,Someone needs to do a Case Study on why Zimbab...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,None,NaN,None,...,7492.0,None,None,None,None,[],"Harare, Zimbabwe",Sat Nov 05 07:40:58 +0000 2011,True,0
91792,2020-03-25 10:02:54+00:00,1.242754e+18,1242753817701019649,"RT @tz_urboyoliver_: If this is true , the Pre...","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,18692.0,None,None,None,None,[],Canada in Jesus name 🙏,Mon Aug 05 16:50:16 +0000 2019,True,1
91793,2020-03-25 10:02:54+00:00,1.242754e+18,1242753817587978241,RT @ActuaryByDay: A bit more detailed context ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,2120.0,None,None,None,None,[],None,Mon Mar 11 06:04:27 +0000 2013,False,1
91794,2020-03-25 10:02:54+00:00,1.242754e+18,1242753817751343104,"RT @vrgoqn: We’re on your side, Dr Amalina. Ou...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,233.0,None,None,None,None,[],None,Mon Sep 06 02:07:00 +0000 2010,True,1


In [17]:
parent_tweet_ids = []
parent_user_ids  = []
for index, row in data.iterrows():
    if row['is_retweet']:
        tweet = row['retweeted_status']
        parent_tweet_ids.append(tweet['id_str'])
        parent_user_ids.append(tweet['user']['id_str'])
    else:
        parent_tweet_ids.append(None)
        parent_user_ids.append(None)
        
data['parent_tweet_id_str'] = parent_tweet_ids
data['parent_user_id_str']  = parent_user_ids

In [18]:
data['has_media'] = pd.notna(data['extended_entities'])

In [19]:
import re

def extract_mentions(x):
    return [str(n.replace("id': ","")) for n in re.findall("id': [0-9]*", str(x))]

data['mentions'] = data['entities'].apply(lambda x: extract_mentions(x))

In [20]:
def extract_hashtags(x):
    return [n.replace("'text': ","") for n in re.findall("'text': '[a-zA-Z0-9_]*'", str(x))]

data['hashtags'] = data['entities'].apply(lambda x: extract_hashtags(x))

In [21]:
selection = ['created_at',                                  # Timestamp for possible time comparisons
             'id_str',                                      # Id of the tweet for collecting replies / retweets
             'hashtags',                                    # Hashtags for coloring the nodes
             'user_id_str',                                 # Identify / define nodes in the network 
             'mentions',                                    # Define interaction-edges for mentions
             'parent_tweet_id_str',
             'parent_user_id_str',                          # Retweets
             'place_coordinates','place_name','place_type',
             'user_defined_location',                       # Keep location/place data for filtering by city 
             'in_reply_to_user_id_str']                     # Define interaction-edges for replies
data_sel  = data[selection]

In [22]:
# construct network

In [23]:
nodes = dict(data[['user_id_str','user_defined_location']].values)

In [24]:
def init_edge(src,trg,edges):
    if (str(src),str(trg)) not in edges: edges[(src,trg)] = Counter({'orig_mentions':0, 'retweets':0, 'replies':0})            

def transform_to_edges(data: pd.DataFrame) -> dict:
    edges = {} # Dict where keys are tuples containing the source and target ids
               # values are Counters with attributes: orig_mentions, retweets, replies
    
    for i,row in data.iterrows():
        src      = row['user_id_str']
        par_id   = row['parent_user_id_str']
        reply_id = row['in_reply_to_user_id_str'] 
        mentions = list(row['mentions'])
        
        # Note that mentions are superset of retweets and replies, thus we 
        # first remove both of them, and then authentic mentions left.
        
        # Case 1. replies
        if reply_id in mentions:
            init_edge(src,reply_id,edges)
            edges[(src,reply_id)]['replies'] += 1
            mentions.remove(reply_id)
        
        # Case 2. retweets
        if par_id is not None:
            init_edge(src,par_id,edges)
            edges[(src,par_id)]['retweets'] += 1
            mentions.remove(par_id) 
            
        # Case 3. mentions
        for trg in mentions:
            init_edge(src,trg,edges)
            edges[(src,trg)]['orig_mentions'] += 1            
        
    return edges

In [25]:
edges = transform_to_edges(data)
print(len(edges), 'edges found in the data set.')

100414 edges found in the data set.


In [26]:
edgelist = [(i[0],i[1],dict(k)) for i,k in edges.items()]
#missing_tweets = list(out_retweet_ids)

In [27]:
nodes

{'3476313672': '08-25-18 7:32PM',
 '148397030': 'Portsmouth NH',
 '912816423529009153': 'Cork, Ireland',
 '1210923069528629248': None,
 '548638630': 'Lagos, Nigeria',
 '923269777308647429': 'Abu Dhabi',
 '1418662837': 'India',
 '375789114': 'Worwai ...We Global 😎',
 '1219007111239282688': None,
 '937862822557466624': None,
 '892286193286688769': None,
 '254589680': 'Riyadh, Kingdom of Saudi Arabi',
 '1009706051854327808': 'Hell➰',
 '375828308': None,
 '1167438893354012675': 'South Africa',
 '1219583333702742016': None,
 '3312981665': 'South West, England',
 '1135860730656415745': '17 dancing queen,, she/her ',
 '1231467376245272576': None,
 '972386818267254784': 'Tema, Ghana',
 '1234075887211106309': None,
 '65940705': 'surrey',
 '742437424350040065': None,
 '969780167932485633': 'Nairobi, Kenya ',
 '1068156118415732736': None,
 '62491620': 'Tsumeb, Namibia',
 '1065144120824512512': None,
 '102784607': 'Scotland',
 '914189502800846849': "don't search ",
 '626627148': 'Atlanta, GA',
 '1

In [28]:
with open('nodelist_ksw_full_372_30min_25-03-2020.json', 'w') as file:
    json.dump(nodes, file, allow_nan=False)
    
with open('edgelist_ksw_full_372_30min_25-03-2020.json', 'w') as file:
    json.dump(edgelist, file, allow_nan=False)